In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/home/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [2]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    df,
                    ACCESS_TOKEN,
                    accountid,
                    df_use_col_name,
                    fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [7]:
import glob
folder_latest_files="/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/"
list_files_to_upload_2020Q4=glob.glob(folder_latest_files+"*.csv")
list_files_to_upload_2020Q4.sort()
print(len(list_files_to_upload_2020Q4))
list_files_to_upload_2020Q4

18


['/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/Control_segment_2020Q4.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_Active0X_H.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_Active0X_L.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_Active0X_M.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_ActiveX12_L.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_ActiveX12_M.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_Lapsed1318_L.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_file_FB/output_2020-10-09/T_Legacy_Lapsed1924_Unknown.csv',
 '/home/sharefolder/Live_Ramp/Quarterly_Update_2020Q4/uploaded_fi

In [8]:
df=pd.read_csv(list_files_to_upload_2020Q4[0])
df.shape

(580124, 3)

In [9]:
df.head(4)

,customer_id_hashed,email_address_hash,agg_seg_regardless_zip
0,17a4ab00dea207b996caa4e126942c67b0e0d53f8c41f2...,1f13ed588000f7bfe7c842b8d8904503e09d2860f0e614...,Control_segment_2020Q4
1,900dc9251d24d03fbbe1d205b9762536d1860e6a183b1f...,413f79c07906cff687de4220a483c53b831a7d238426e8...,Control_segment_2020Q4
2,24595226987543c9037f8a5be53304621516c2fddf8579...,287446d41df662952ee2ca98a323e39dc98b23926c300d...,Control_segment_2020Q4
3,44a79ad9932ab655715def34009d646f9a4526cf002a53...,388a32df1bbdd1d083dbee6f536d66bb7c32681a1747d9...,Control_segment_2020Q4


In [10]:
df['agg_seg_regardless_zip'].unique()

array(['Control_segment_2020Q4'], dtype=object)

In [11]:
for file in list_files_to_upload_2020Q4:
    for fb_account_id in list_account_id:
        audience_name = os.path.basename(file).split(".")[0]
        description="2020Q4 audience to be refreshed: "+os.path.basename(file).split(".")[0]+"_"+str(datetime.datetime.now().date())
        df = pd.read_csv(file,dtype=str)
        creatfbaudience(AudienceName="aud_"+audience_name,
                        AudienceDescription=description,
                        df=df,
                        df_use_col_name='email_address_hash',
                        ACCESS_TOKEN=jian_token,
                        accountid=fb_account_id)

Audience ID: 23845696138640265
2020-10-12 11:54:21.416416 aud_Control_segment_2020Q4 uploaded 580124
Audience ID: 23846059927890632
2020-10-12 11:55:17.588823 aud_Control_segment_2020Q4 uploaded 580124
Audience ID: 23845696145270265
2020-10-12 12:04:03.464064 aud_T_Legacy_Active0X_H uploaded 5420744
Audience ID: 23846060022670632
2020-10-12 12:12:49.662844 aud_T_Legacy_Active0X_H uploaded 5420744
Audience ID: 23845696291100265
2020-10-12 12:13:53.300058 aud_T_Legacy_Active0X_L uploaded 583080
Audience ID: 23846060084000632
2020-10-12 12:14:46.494586 aud_T_Legacy_Active0X_L uploaded 583080
Audience ID: 23845696305510265
2020-10-12 12:20:54.968861 aud_T_Legacy_Active0X_M uploaded 3394596
Audience ID: 23846060113100632
2020-10-12 12:25:36.273789 aud_T_Legacy_Active0X_M uploaded 3394596
Audience ID: 23845696357760265
2020-10-12 12:28:13.292273 aud_T_Legacy_ActiveX12_L uploaded 1623504
Audience ID: 23846060144470632
2020-10-12 12:30:34.506230 aud_T_Legacy_ActiveX12_L uploaded 1623504
Audien